In [7]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Retrieving option chain data

In [8]:

def get_options_chain(ticker_symbol, expiration_date):
    """
    Fetches the options chain for the given ticker and expiration date.

    Parameters:
    ticker_symbol (str): The ticker symbol for the stock.
    expiration_date (str): The expiration date for the options chain.

    Returns:
    pandas.DataFrame: The options chain for the given ticker and expiration date.
    """
    # Fetch the data for the ticker
    ticker_data = yf.Ticker(ticker_symbol)

    # Fetch the options chain for the chosen expiration date
    options_chain = ticker_data.option_chain(expiration_date)

    return options_chain




In [12]:
oc = get_options_chain('AAPL', '2024-03-28')

In [32]:
calls = pd.DataFrame(oc.calls)

In [36]:
aapl190c = calls.loc[calls['strike'] == 190.0]
r = 1 + 0.055
S = 181.56
K = 190
T = 1/12
u

,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency
12,AAPL240328C00190000,2024-02-20 17:44:27+00:00,190.0,1.8,1.77,1.8,-0.44,-19.730946,373.0,1793,0.214974,False,REGULAR,USD


## Using the binomial tree modeel to calculate the delta

In [45]:
def american_call_binomial(S, K, r, sigma, T):
    """
    Prices an American call option using a two-period binomial tree and calculates delta.

    Parameters:
    S (float): Current stock price
    K (float): Strike price of the option
    r (float): Risk-free interest rate for one period
    sigma (float): Implied Volatility of the stock
    T (float): Time to expiration
    
    Returns:
    C (float): Price of the American call option
    delta (float): Delta of the option
    """
    # Define up and down movement factors
    u = np.exp(sigma* np.sqrt(T/2))
    d = 1/u

    # Calculate risk-neutral probabilities
    p = (r - d) / (u - d)
    
    # Calculate the option value at terminal nodes
    Cuu = max(0, u**2 * S - K)
    Cud = max(0, u * d * S - K) # Since it's an American option, we consider the possibility of early exercise
    Cdd = max(0, d**2 * S - K)
    
    # Calculate the option value at the first nodes
    Cu = (p * Cuu + (1 - p) * Cud) / r
    Cd = (p * Cud + (1 - p) * Cdd) / r
    
    # Calculate the option value now
    C = max(S - K, (p * Cu + (1 - p) * Cd) / r)
    
    # Calculate delta
    delta = (Cu - Cd) / ((u - d) * S)
    
    return C, delta

# Example use on aapl 180c:
#AAPL current price
S = yf.Ticker('AAPL').history(period='1d').iloc[0]['Close']
#get options chain
AppleOC = get_options_chain('AAPL', '2024-03-28')
#calls
calls = pd.DataFrame(AppleOC.calls)
#180 strike call
aapl190c = calls.loc[calls['strike'] == 180.0]
k = 180
r = 1 + 0.055
T = 1/12
#implied volatility
IV = aapl190c['impliedVolatility'].astype(float).iloc[0]

print(f'S = {S}, K = {k}, r = {r}, T = {T}, IV = {IV}')
option_price, option_delta = american_call_binomial(S=S, K=k, r=r, sigma=IV, T=T)


print(f"The price of the option is: {option_price:.2f}")
print(f"The delta of the option is: {option_delta:.2f}")


S = 184.3699951171875, K = 180, r = 1.055, T = 0.08333333333333333, IV = 0.22901161621093752
The price of the option is: 22.71
The delta of the option is: 1.05


c:\Users\olive\anaconda3\envs\PortOptEnv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [47]:
import numpy as np
import yfinance as yf
import pandas as pd

def american_call_binomial(S, K, r, sigma, T):
    """
    Prices an American call option using a two-period binomial tree and calculates delta.

    Parameters:
    S (float): Current stock price
    K (float): Strike price of the option
    r (float): Risk-free interest rate for one period
    sigma (float): Implied Volatility of the stock
    T (float): Time to expiration
    
    Returns:
    C (float): Price of the American call option
    delta (float): Delta of the option
    """
    # Define up and down movement factors
    u = np.exp(sigma * np.sqrt(T / 2))
    d = 1 / u

    # Calculate risk-neutral probabilities
    p = (np.exp(r * T / 2) - d) / (u - d)
    
    # Calculate the option value at terminal nodes
    Cuu = max(0, u**2 * S - K)
    Cud = max(0, u * d * S - K)  # Since it's an American option, we consider the possibility of early exercise
    Cdd = max(0, d**2 * S - K)
    
    # Calculate the option value at the first nodes
    Cu = np.exp(-r * T / 2) * (p * Cuu + (1 - p) * Cud)
    Cd = np.exp(-r * T / 2) * (p * Cud + (1 - p) * Cdd)
    
    # Calculate the option value now
    C = np.exp(-r * T / 2) * (p * Cu + (1 - p) * Cd)
    
    # Calculate delta
    delta = (Cu - Cd) / ((u - d) * S)
    
    return C, delta
# Example use on march 28 aapl 180c:
#AAPL current price
S = yf.Ticker('AAPL').history(period='1d').iloc[0]['Close']
#get options chain
AppleOC = get_options_chain('AAPL', '2024-03-28')
#calls
calls = pd.DataFrame(AppleOC.calls)
#180 strike call
aapl190c = calls.loc[calls['strike'] == 180.0]
k = 180
r = 0.055
T = 1/12
#implied volatility
IV = aapl190c['impliedVolatility'].astype(float).iloc[0]

print(f'S = {S}, K = {k}, r = {r}, T = {T}, IV = {IV}')
option_price, option_delta = american_call_binomial(S=S, K=k, r=r, sigma=IV, T=T)


print(f"The price of the option is: {option_price:.2f}")
print(f"The delta of the option is: {option_delta:.2f}")


S = 184.3699951171875, K = 180, r = 0.055, T = 0.08333333333333333, IV = 0.22901161621093752
The price of the option is: 8.05
The delta of the option is: 0.66


c:\Users\olive\anaconda3\envs\PortOptEnv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [49]:
# Example use on jan 17 2025 aapl 200c:
#AAPL current price
S = yf.Ticker('AAPL').history(period='1d').iloc[0]['Close']
#get options chain
AppleOC = get_options_chain('AAPL', '2025-01-17')
#calls
calls = pd.DataFrame(AppleOC.calls)
#180 strike call
aapl200c = calls.loc[calls['strike'] == 200.0]
k = 200
r = 0.055
T = 11/12
#implied volatility
IV = aapl200c['impliedVolatility'].astype(float).iloc[0]

print(f'S = {S}, K = {k}, r = {r}, T = {T}, IV = {IV}')
option_price, option_delta = american_call_binomial(S=S, K=k, r=r, sigma=IV, T=T)


print(f"The price of the option is: {option_price:.2f}")
print(f"The delta of the option is: {option_delta:.2f}")


c:\Users\olive\anaconda3\envs\PortOptEnv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


S = 184.3699951171875, K = 200, r = 0.055, T = 0.9166666666666666, IV = 0.25659923095703124
The price of the option is: 16.27
The delta of the option is: 0.49


In [50]:
def american_put_binomial(S, K, r, sigma, T):
    """
    Prices an American put option using a two-period binomial tree and calculates delta.

    Parameters:
    S (float): Current stock price
    K (float): Strike price of the option
    r (float): Risk-free interest rate for one period
    sigma (float): Implied Volatility of the stock
    T (float): Time to expiration
    
    Returns:
    P (float): Price of the American put option
    delta (float): Delta of the option
    """
    # Define up and down movement factors
    u = np.exp(sigma * np.sqrt(T / 2))
    d = 1 / u

    # Calculate risk-neutral probabilities
    p = (np.exp(r * T / 2) - d) / (u - d)
    
    # Calculate the option value at terminal nodes
    Puu = max(0, K - u**2 * S)
    Pud = max(0, K - u * d * S)  # Since it's an American option, we consider the possibility of early exercise
    Pdd = max(0, K - d**2 * S)
    
    # Calculate the option value at the first nodes
    Pu = np.exp(-r * T / 2) * (p * Puu + (1 - p) * Pud)
    Pd = np.exp(-r * T / 2) * (p * Pud + (1 - p) * Pdd)
    
    # Calculate the option value now
    P = np.exp(-r * T / 2) * (p * Pu + (1 - p) * Pd)
    
    # Calculate delta
    delta = (Pu - Pd) / ((u - d) * S)
    
    return P, delta

In [51]:
# Example use on AAPL 200 put:
# AAPL current price
S = yf.Ticker('AAPL').history(period='1d').iloc[0]['Close']
# Get options chain
AppleOC = yf.Ticker('AAPL').option_chain('2025-01-17')
# Puts
puts = pd.DataFrame(AppleOC.puts)
# 180 strike put
aapl200p = puts.loc[puts['strike'] == 200.0]
k = 200
# Assuming the risk-free rate is 5.5% annually, expressed as a decimal
r = 0.055
T = 11/12
# Implied volatility
IV = aapl200p['impliedVolatility'].astype(float).iloc[0]

print(f'S = {S}, K = {k}, r = {r}, T = {T}, IV = {IV}')
option_price, option_delta = american_put_binomial(S=S, K=k, r=r, sigma=IV, T=T)

print(f"The price of the option is: {option_price:.2f}")
print(f"The delta of the option is: {option_delta:.2f}")

S = 184.3699951171875, K = 200, r = 0.055, T = 0.9166666666666666, IV = 0.15451895294189455
The price of the option is: 15.00
The delta of the option is: -0.59


c:\Users\olive\anaconda3\envs\PortOptEnv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [52]:
# Example use on AAPL 180 put:
# AAPL current price
S = yf.Ticker('AAPL').history(period='1d').iloc[0]['Close']
# Get options chain
AppleOC = yf.Ticker('AAPL').option_chain('2024-03-28')
# Puts
puts = pd.DataFrame(AppleOC.puts)
# 180 strike put
aapl200p = puts.loc[puts['strike'] == 180.0]
k = 180
# Assuming the risk-free rate is 5.5% annually, expressed as a decimal
r = 0.055
T = 1/12
# Implied volatility
IV = aapl200p['impliedVolatility'].astype(float).iloc[0]

print(f'S = {S}, K = {k}, r = {r}, T = {T}, IV = {IV}')
option_price, option_delta = american_put_binomial(S=S, K=k, r=r, sigma=IV, T=T)

print(f"The price of the option is: {option_price:.2f}")
print(f"The delta of the option is: {option_delta:.2f}")

S = 184.3699951171875, K = 180, r = 0.055, T = 0.08333333333333333, IV = 0.1834798449707031
The price of the option is: 2.04
The delta of the option is: -0.31


c:\Users\olive\anaconda3\envs\PortOptEnv\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
